In [ ]:
import numpy as np
import cv2
import os
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

# Load the trained model
model = keras.models.load_model('fer_model.h5')

# Define the test dataset directory
test_directory = "Test"

# Classes for facial emotions
classes = ["0", "1", "2", "3", "4", "5", "6"]

# Load test data
test_data = []
test_labels = []
img_size = 224
for category in classes:
    path = os.path.join(test_directory, category)
    class_num = classes.index(category)
    for img in os.listdir(path):
        try:
            img_array = cv2.imread(os.path.join(path, img))
            new_array = cv2.resize(img_array, (img_size, img_size))
            test_data.append(new_array)
            test_labels.append(class_num)
        except Exception as e:
            pass

# Convert test data to numpy array
test_data = np.array(test_data).reshape(-1, img_size, img_size, 3)
test_data = test_data / 255.0  # Normalize the data

# Predict using the model
predictions = model.predict(test_data)
predicted_classes = np.argmax(predictions, axis=1)

# Calculate confusion matrix
cm = confusion_matrix(test_labels, predicted_classes)

# Convert confusion matrix to percentage
cm_percentage = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Plot confusion matrix with percentage values
plt.figure(figsize=(10, 8))
sns.heatmap(cm_percentage, annot=True, fmt=".2f", cmap="Blues", xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Classification report
print(classification_report(test_labels, predicted_classes, target_names=classes))
